# Chess-Bot

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

2022-12-22 02:04:14.252705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 02:04:14.345377: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-22 02:04:14.347978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 02:04:14.347992: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

## Environment

In [2]:
cb.environment.create()

Dataset archive file found at /home/whumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/whumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/whumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/whumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [ ]:
x_train, x_val = cb.dataset.pos2vec_dataset()
pos2vec = cb.model.train_pos2vec(x_train, x_val)

2022-12-22 02:04:43.418133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-22 02:04:43.418303: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 02:04:43.418363: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-22 02:04:43.418414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-22 02:04:43.418436: W tensorf

Training layer 1/5
40000/40000 [==============================] - 210s 5ms/step - loss: 0.2749 - accuracy: 0.0140 - val_loss: 0.1031 - val_accuracy: 0.0050 - lr: 0.0050
Training layer 2/5
  453/40000 [..............................] - ETA: 3:57 - loss: 0.1898 - accuracy: 4.4150e-05

In [ ]:
import numpy as np

ex = np.reshape(x_train[0][0][0], (1, 773))

print("ex")
print(ex)

print("enc")
print(len(pos2vec(ex)[0]))

In [ ]:
temp = cb.dataset.pos2vec_dataset()

In [ ]:
print(len(temp[0]))

In [ ]:
# example = np.reshape(x_train[0][0][0], (1, 773))
# print(example)
# encoded = pos2vec.encoder(example)
# print(encoded)
# print(pos2vec.decoder(encoded))

## Sandbox

In [ ]:
import numpy as np
import pandas as pd

win = pd.read_pickle("./data/win.pkl")
print(win.head())
print(len(win))

win_val = pd.read_pickle("./data/win_val.pkl")
print(win_val.head())
print(len(win_val))

## TODO
- Parquet, read w/ fastparquet write w/ pyarrow